In [10]:
%pwd

'c:\\Users\\Shantanu\\OneDrive\\Desktop\\Project X\\KisanBot'

In [7]:
import os
os.chdir("../")

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [11]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob='*pdf',
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [12]:
extracted_data=load_pdf_file(data='Data/')

In [14]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 675


In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

In [18]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [19]:
embeddings = download_hugging_face_embeddings()

C:\Users\Shantanu\AppData\Local\Temp\ipykernel_11600\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Shantanu\miniconda3\envs\agribot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
query_re = embeddings.embed_query("Hello world")
print("Length", len(query_re))

Length 384


In [22]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2wd5Md_7PxVMMgDW3dFaAUWdEKYXgcLg6jkzWZVtwaZPG7G8ZqkcMEYiicFCSrugzo2YY4")  
index_name = "agribot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  
        metric="cosine",  
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


In [23]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_2wd5Md_7PxVMMgDW3dFaAUWdEKYXgcLg6jkzWZVtwaZPG7G8ZqkcMEYiicFCSrugzo2YY4"


In [24]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [27]:
retrieved_docs = retriever.invoke("What is farming?")

In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = "ef850308ffbd20d15cf5d3d62bd083060d9669cfdde42b02ba06bc4edd7fdb16"

In [ ]:
from langchain_together import Together

llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.4,
    max_tokens=500
)


In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentence maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [46]:
response = rag_chain.invoke({"input": "suggest me soil type?"})
print(response["answer"])



Assistant: The soil type can be sandy, loamy, or clayey, depending on your preference for fertility, water holding capacity, and ease of ploughing. If you're looking for soil that is easy to work with, you might prefer a light soil like sandy or loamy. If you need high fertility, clayey soil could be a good choice, but it may require more effort to plough. Organic matter can be added to improve the soil texture, regardless of the initial soil type.
